In [1]:
# imports
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from dotenv import load_dotenv
import os
from typing import Callable

load_dotenv()
raw_data_folder = os.getenv("rawDataDirectory")
data_folder = os.getenv("dataFolder")
privateLatStr = os.getenv("privateLat")
privateLonStr = os.getenv("privateLon")
privateRadStr = os.getenv("privateRad")

assert (
    privateLatStr is not None
    and privateLonStr is not None
    and privateRadStr is not None
)

privateLat = float(privateLatStr)
privateLon = float(privateLonStr)
privateRad = float(privateRadStr)

from util.clean_file import filter_by_distance

filterFunction: Callable[[float, float], bool] = lambda lat, lon: filter_by_distance(
    (lat, lon), (privateLat, privateLon, privateRad)
)

In [2]:
init = pd.read_csv(
    f"{raw_data_folder}/data_V2",
    delimiter=";",
    header=0,
    dtype={
        "finePermission": bool,
        "foreGround": bool,
        "priority": int,
        "user": str,
        "latitude": float,
        "longitude": float,
        "time": int,
        "accuracy": float,
    },
)
init["time"] = pd.to_datetime(init["time"], unit="ms")  # type: ignore
print(init.shape)
data = init[
    init.apply(
        lambda row: filterFunction(row["latitude"], row["longitude"]), # type: ignore
        axis=1,
    )
]
print(data.shape)

(2584, 8)
(2413, 8)


In [3]:
# data manipulation
data['date'] = data['time'].dt.date
groups=data.groupby('date')
print(groups.size())

date
2024-10-15     315
2024-10-16     529
2024-10-18    1569
dtype: int64


C:\Users\MilanSchollier\AppData\Local\Temp\ipykernel_10648\4101109088.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data['time'].dt.date


In [4]:
# maakt voor ieder device en per dag een groep aan
for date, group in groups:
    map = folium.Map(
        location=[group["latitude"].mean(), group["longitude"].mean()], zoom_start=13
    )

    # voor masten(comment out if not needed)
    # marker_cluster=MarkerCluster().add_to(map)
    # for coord in coordinates:
    #     folium.Marker([coord[1], coord[0]]).add_to(marker_cluster)

    # 2 trails
    fine_balanced = []
    fine_high = []
    coarse_balanced = []
    coarse_high = []
    for index, row in group.iterrows():
        if row["finePermission"]:
            if row["priority"] == 100:
                fine_high.append((row["latitude"], row["longitude"]))
            elif row["priority"] == 102:
                fine_balanced.append((row["latitude"], row["longitude"]))
            else:
                print("fine but no priority: " + row["priority"])
        else:
            if row["priority"] == 100:
                coarse_high.append((row["latitude"], row["longitude"]))
            elif row["priority"] == 102:
                coarse_balanced.append((row["latitude"], row["longitude"]))
            else:
                print("coarse but no priority: " + row["priority"])
    print("size of fine_high: " + str(len(fine_high)))
    print("size of fine_balanced: " + str(len(fine_balanced)))
    print("size of coarse_high: " + str(len(coarse_high)))
    print("size of coarse_balanced: " + str(len(coarse_balanced)))
    if len(fine_high)>0:
        folium.PolyLine(
            locations=fine_high,
            color="#FF0000",
            tooltip="fine grained location and high accuracy",
        ).add_to(map)
    if len(fine_balanced)>0:
        folium.PolyLine(
            locations=fine_balanced,
            color="#FF0000",
            dash_array="10",
            tooltip="fine grained location and balanced accuracy",
        ).add_to(map)
    if len(coarse_high)>0:
        folium.PolyLine(
            locations=coarse_high,
            color="#0000FF",
            tooltip="coarse grained location and high accuracy",
        ).add_to(map)
    if len(coarse_balanced)>0:
        folium.PolyLine(
            locations=coarse_balanced,
            color="#0000FF",
            dash_array="10",
            tooltip="coarse grained location and balanced accuracy",
        ).add_to(map)
    map.save(f"{data_folder}/v2/{date}.html")

size of fine_high: 164
size of fine_balanced: 112
size of coarse_high: 25
size of coarse_balanced: 14
size of fine_high: 437
size of fine_balanced: 59
size of coarse_high: 26
size of coarse_balanced: 7
size of fine_high: 1163
size of fine_balanced: 322
size of coarse_high: 63
size of coarse_balanced: 21
